In [1]:
!ls

sample_data


In [2]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          40G   17G   21G  45% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
tmpfs           6.4G  4.0K  6.4G   1% /var/colab
/dev/sda1        46G   20G   26G  44% /etc/hosts
shm             6.0G     0  6.0G   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [3]:
from google.colab import files
uploaded = files.upload()

Saving gdown.pl-master.zip to gdown.pl-master.zip


In [4]:
!unzip gdown.pl-master.zip

Archive:  gdown.pl-master.zip
dfd6dc910a38a42d550397bb5c2335be2c4bcf54
   creating: gdown.pl-master/
 extracting: gdown.pl-master/.gitattributes  
  inflating: gdown.pl-master/LICENSE  
  inflating: gdown.pl-master/README.md  
  inflating: gdown.pl-master/gdown.pl  


In [5]:
!gdown.pl-master/gdown.pl https://drive.google.com/file/d/1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL/view?usp=sharing Neural_Nets.zip

Cannot open cookies file ‘gdown.cookie.temp’: No such file or directory
--2018-12-22 07:34:10--  https://docs.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL&export=download
Resolving docs.google.com (docs.google.com)... 74.125.132.100, 74.125.132.101, 74.125.132.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.132.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Neural_Nets.zip’

Neural_Nets.zip         [ <=>                ]   3.18K  --.-KB/s    in 0s      

2018-12-22 07:34:10 (28.3 MB/s) - ‘Neural_Nets.zip’ saved [3252]

--2018-12-22 07:34:10--  https://docs.google.com/uc?export=download&confirm=aruh&id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
Resolving docs.google.com (docs.google.com)... 74.125.124.102, 74.125.124.100, 74.125.124.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.124.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-

In [0]:
!unzip Neural_Nets.zip

In [7]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
!pip install keras
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In [0]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10#experiment with the batch size

In [0]:
def generator(source_path, folder_list, batch_size):
    x = 30
    y = 100
    z = 100
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = list(range(30))#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size)# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imresize(image, (y,z), interp='bilinear')
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0]#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1]#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2]#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        batch_data = np.zeros((len(t) - batch_size*num_batches,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((len(t) - batch_size*num_batches,5)) # batch_labels is the one hot representation of the output
        # write the code for the remaining data points which are left after full batches
        for folder in range(batch_size*num_batches, len(t)): # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes
                image = imread(source_path+'/'+ t[folder].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                image = imresize(image, (y,z), interp='bilinear')

                batch_data[folder - batch_size*num_batches,idx,:,:,0] = image[:,:,0]#normalise and feed in the image
                batch_data[folder- batch_size*num_batches,idx,:,:,1] = image[:,:,1]#normalise and feed in the image
                batch_data[folder-batch_size*num_batches,idx,:,:,2] = image[:,:,2]#normalise and feed in the image

            batch_labels[folder-batch_size*num_batches, int(t[folder].strip().split(';')[2])] = 1
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do



In [32]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


In [33]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, LSTM
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here


model = Sequential()
# define CNN model
model.add(TimeDistributed(Conv2D(32, kernel_size=(3), strides=(2, 2), input_shape=(100, 100,3))))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
# define LSTM model
model.add(LSTM(128, return_sequences=True))
model.add(Dense(5, activation='softmax'))

# model summary
model.summary()

ValueError: ignored

In [0]:
optimiser = optimizers.SGD(lr=0.01, clipnorm=1.)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

In [0]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [0]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [0]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [27]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


ResourceExhaustedError: ignored